In [ ]:
# Imports ||||Run on Restart|||

from utils import get_embeddings_from_state_dict, seed_everything

import numpy as np

# Statistics Silo

This is where you pipe in the data generated by the HBBS or ABBS to compare results and get statistics.

In [ ]:
# --- Parameters to set before running the script ---

BASE_RANDOM_STATE = 42  # Initial seed for reproducibility of the entire sequence of experiments
DIR = "Datasets" # Path to the directory containing the models in relation to this 
EXPERIMENT = "Kingdom1" # Name of the experiment to test
NUM_RUNS = 10 # Number of runs of experiment
RUN_NUMBER = 5 # Used for t-SNE and other analyzes that take a single run


## t-SNE Analysis

### Euclidian distance analysis on a whim
a thing to compare euclidian distances (that annoying copilot autocomplete thing suggested it below code I was writing for t-SNE analysis and I got intrigued and let it finish the code. after editing the code to be actually useful I did some research. seems like this analysis could actually be useful for my project).

In [ ]:
# --- Generate a dictionary of lists containing the results of euclidian distance analysis of all embeddings of the experiment specified above ---

# Set the random seed for reproducibility
seed_everything(BASE_RANDOM_STATE, silent=True)


# Initialize a dictionary to store the results
Euclidian_results = {"baseline_euclidean_distance": [],
                    "glylm_euclidean_distance": [],
                    "baseline_glylm_euclidean_distance": [],
                    "baseline_runs_euclidean_distance": []}

print(f"Calculating the Euclidean distance of the embeddings of the experiment {EXPERIMENT} with {NUM_RUNS} runs...")
print()

for runs in range(NUM_RUNS):
    # --- load the embeddings ---
    # Construct paths to the relevant state dictionaries
    other_run = runs + 2 if runs < NUM_RUNS - 1 else 1 
    baseline_trainable_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{runs+1}_state_dict.pth"
    baseline_trainable_comp_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{other_run}_state_dict.pth"
    baseline_fixed_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_fixed_run_{runs+1}_state_dict.pth"
    infused_trainable_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_trainable_run_{runs+1}_state_dict.pth"
    infused_fixed_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_fixed_run_{runs+1}_state_dict.pth"

    # Load the embeddings from the state dictionaries
    baseline_emb = get_embeddings_from_state_dict(baseline_trainable_path)
    baseline_comp_emb = get_embeddings_from_state_dict(baseline_trainable_comp_path)
    fixed_baseline_emb = get_embeddings_from_state_dict(baseline_fixed_path)
    glylm_emb = get_embeddings_from_state_dict(infused_fixed_path)
    trained_glylm_emb = get_embeddings_from_state_dict(infused_trainable_path)

    # Calculate the Euclidean distance between the embeddings
    baseline_euclidean_dist = np.linalg.norm(baseline_emb - fixed_baseline_emb)
    glylm_euclidean_dist = np.linalg.norm(glylm_emb - trained_glylm_emb)
    baseline_glylm_euclidean_dist = np.linalg.norm(baseline_emb - glylm_emb)
    # Calculate the Euclidean distance between the embeddings of two runs
    baseline_runs_euclidean_dist = np.linalg.norm(baseline_emb - baseline_comp_emb)

    # add the euclidean distance of the current run to the list of results
    Euclidian_results["baseline_euclidean_distance"].append(baseline_euclidean_dist)
    Euclidian_results["glylm_euclidean_distance"].append(glylm_euclidean_dist)
    Euclidian_results["baseline_glylm_euclidean_distance"].append(baseline_glylm_euclidean_dist)
    Euclidian_results["baseline_runs_euclidean_distance"].append(baseline_runs_euclidean_dist)

# --- Calculate the average of the euclidean distances ---
baseline_euclidean_dist_avg = np.mean(Euclidian_results["baseline_euclidean_distance"])
glylm_euclidean_dist_avg = np.mean(Euclidian_results["glylm_euclidean_distance"])
baseline_glylm_euclidean_dist_avg = np.mean(Euclidian_results["baseline_glylm_euclidean_distance"])
baseline_runs_euclidean_dist_avg = np.mean(Euclidian_results["baseline_runs_euclidean_distance"])

# Calculate the standard deviation of the euclidean distances
baseline_euclidean_dist_std = np.std(Euclidian_results["baseline_euclidean_distance"])
glylm_euclidean_dist_std = np.std(Euclidian_results["glylm_euclidean_distance"])
baseline_glylm_euclidean_dist_std = np.std(Euclidian_results["baseline_glylm_euclidean_distance"])
baseline_runs_euclidean_dist_std = np.std(Euclidian_results["baseline_runs_euclidean_distance"])

# Print the average euclidean distance values with their standard deviation
print(f"Average Baseline (fixed to trainable) Euclidean Distance: {baseline_euclidean_dist_avg} ± {baseline_euclidean_dist_std}")
print(f"Average GlyLM (fixed to trainable) Euclidean Distance: {glylm_euclidean_dist_avg} ± {glylm_euclidean_dist_std}")
print(f"Average Baseline-GlyLM [trainable] Euclidean Distance: {baseline_glylm_euclidean_dist_avg} ± {baseline_glylm_euclidean_dist_std}")
print(f"Average Baseline [trainable] (comparing runs) Euclidean Distance: {baseline_runs_euclidean_dist_avg} ± {baseline_runs_euclidean_dist_std}")




Baseline (fixed to trainable) Euclidean Distance: 1282.7105712890625
GlyLM (fixed to trainable) Euclidean Distance: 27.04410743713379
Baseline-GlyLM [trainable] Euclidean Distance: 1216.6019287109375
Baseline [trainable] (run 1 to run 2) Euclidean Distance: 1282.34619140625